## Quick Abc to midi converter
- http://www.mandolintab.net/abcconverter.php


## Major and Minor
- https://www.youtube.com/watch?v=fKpUBsn_jmA
- help create mood and atmossphere of music

In [1]:
import os
import json
import numpy as np
import pandas as pd
import sys
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
BATCH_SIZE = 16
N_GRAM = 64
MODEL_DIR = './music_model/LSTMSkipConnection_many2many_128_256_128_layers_lstm/'
DATA_PATH = './data/jig_hornpipes_cleaned.txt'
MODEL_TYPE = 'Default'#unuse for now
BATCH_SIZE = 1
SEQ_LENGTH = 1

In [3]:
with open(os.path.join(MODEL_DIR, 'model_dictionary.json')) as f:
    vocab_map = json.load(f)

n_vocab = len(vocab_map['idx2char'])

print('n_vocab', n_vocab)

n_vocab 67


In [4]:
from models import MusicModel
# model = MusicModel(n_vocab).TwoLayerLSTM(batch_input_shape=(BATCH_SIZE, 1))
# model = MusicModel(n_vocab).TwoLayerLSTM(batch_input_shape=(BATCH_SIZE, SEQ_LENGTH))

# model = MusicModel(n_vocab).LayersRNNGeneric(batch_input_shape=(BATCH_SIZE, SEQ_LENGTH),
#                                             layers=['lstm', 'lstm', 'lstm'], 
#                                              layers_size=[128, 256, 128], drop_rate=0.4)

# model = MusicModel(n_vocab).LSTMSkipConnection(batch_input_shape=(BATCH_SIZE, SEQ_LENGTH),
#                                                                 layers=[128, 256, 128],
#                                                                 emb_dim=256,drop_rate=0.2)

model = MusicModel(n_vocab).LSTMSkipConnection(batch_input_shape=(BATCH_SIZE, SEQ_LENGTH),
                    layers=[128, 256, 128],
                    emb_dim=256,drop_rate=0.3)
print('\n',model.summary())
model.load_weights(MODEL_DIR+'/model_weight/'+"Weights_{}.h5".format(60))

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (1, 1)               0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (1, 1, 256)          17152       input[0][0]                      
__________________________________________________________________________________________________
drop_layer1 (Dropout)           (1, 1, 256)          0           embedding[0][0]                  
__________________________________________________________________________________________________
lstm_layer0 (LSTM)              (1, 1, 128)          197120      drop_layer1[0][0]                
__________________________________________________________________________________________________
lstm_layer

In [54]:
def generate_sequence(model, vocab_map, initial_index=0, seq_length=300):
    sequence_index = [initial_index]
    n_vocab = len(vocab_map['idx2char'])
    prev_note = '\n'
    stop_note = ['\n', '%']
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
#         print(batch)
        predicted_probs = model.predict_on_batch(batch).ravel()
#         print(predicted_probs.shape)
        
        sample = np.random.choice(range(n_vocab), size = 1, p = predicted_probs)
#         while vocab_map['idx2char'][str(sample[0])] in ['\n', '%']:
#         print(sample[0])
#         if  vocab_map['idx2char'][str(sample[0])] in ['\n','%', '\\']:
#             continue
        while vocab_map['idx2char'][str(sample[0])] in stop_note and prev_note in stop_note:
            sample = np.random.choice(range(n_vocab), size = 1, p = predicted_probs)
    
        prev_note = vocab_map['idx2char'][str(sample[0])]
        sequence_index.append(sample[0]) 
        
    str_seq = seq = ''.join(vocab_map['idx2char'][str(c)] for c in sequence_index)
    return str_seq
    

In [80]:
import random
initial_index = np.random.randint(0, n_vocab, size=(1,))[0]
music_abc = generate_sequence(model, vocab_map, initial_index=initial_index, seq_length=300)

print(music_abc)

4|"A"AAA eAA|"D"AdA fAa|"G"gab g3|"G"bge "D"agf|"E"gfe edB|
"A"A2A ABe|"A"ceA age|"A7"cea gfe|"D"d3 -d||
|
"E7"cde BdB|"Em"efe dcB|"A"ecA "A"Ace|"D"def "E7"e2=g|"A"aec a2:|
 [2"A"Ace ag=g||
"D"a2f a2f|"A"eac A2e|"Bm"gfe "B7"fed|"E7"e3 e2:|
P:B
f/2g/2|"A"aec aec|"D"d2b a2f|"A"e2c ABc|"D"d2e fga|
"A7"b


In [5]:
#header info: http://trillian.mit.edu/~jc/music/abc/doc/ABCtut_Headers.html#I_Key
header = '''
X: 1
T:AbcRnn
% abcRnn
M:4/4
K:K
'''

selected = '''
^D/2A/2B/2A/2A F2F/2G/2|
Aca aag|f/2a/2g/2f/2d/2 e3:|
\
A/2g/2f/2g/2|
f/2a3/2
e/2A/2 f/2g/2g/2f/2d/2^G/2|
"D"f2d/2d/2 "Bm"a3/2f/2a/2|"Em"g3 "E7/c+"e2d|"A7/c+"e2e "F/f+"d2c|"Bm"B2B A2^g|
"B7"a2f f2f|"E"g2e "B7"B2d|"B7"e2f g2g|"B7"a3 a2f|
"D7"d3 -c=cd|"G"B2G GBd|"C"e2e c2e|"Am"a3 "G7"g2d|"C"g3 z2:|
P:C


-----------------------------------------------------------------------------
_ef|"A"a^ag "D"fed|"F"ecA "G"Bcd|"Am"e3 -"D7"d2||
K:D
"D"AFA "G"Bcd|"D"Adf "G"gab|"D"agf def|"A7"edc A2a|"A7"afd e2c|
"D"d3 d2f|"D"A2f "A/e"c2e|"Bm"dcd "E7"e3|"A"cBc def|
"Bm"dfB BcB|"A"cde "E7"e2A|[1"A"A3 -A2||
 [2"A"A3 A2G/2F/2|E/2A/2e/2 "D"dcd|"A"e3 e2c|"A"A3 -A7||
K
K:D
"D"[2F2A "A"ABc|"Bm"d2d "D
'''

selected = '''
X:1
M:3/4
L:1/16
K:G
[E,,4E4B,4G4E,4G,4] | [G4E,4E4G,4B,4E,,4]>F4 [E,,4E4G,4E,4] | [D,8A8A,8F8D,,8] [D4F4A4] | [D4F4A4C4]>[B,4D4G4] [C4A4F4A,4] | [B,8B8G8G,8] [G,,4G4D4B4G,4B,4] | [B4G,4G4D4B,4G,,4]>A4 [G,,,4G4B,4G,,4] |
[F,,8^c8C8^A8F,,,8] [F4A4c4] | [F4^A4^c4E4]>[D4F4B4] [E4c4A4C4] | [D8d8B8B,8] d4 | [D4B,4G4] g4 [e4E4] |
[d4A,4] c4 [^D4c4F4] | D4 a4 [F4c4] | G4 B4 [G,4B4] | E,4 G4 [E4G4] |
C4 E4 [E4C,4] | C,,4 G,4 E,4 | [B,,,8^D,8B,8B,,8] [e'4e4] | [G,,4B,4E,4]>[^d'4d4] [e'4e4] |
[A,,8E8F8C8] [^A,,4E4F4^C4] | [F4B,,4^D4A,4]>G4 A2G2 | [F4C,4G,4] E4 [B,,2e'2e2]A,,2 | [G,,4B,4E,4]>[^d'4d4] [^G,,4e4e'4] |
[A,,8E8F8C8] [^A,,4E4F4^C4] | [F4B,,4A,4^D4B,,,4]>A4 G2F2 | [E,,8G,8E8E,8] [B,,4B,4B4B,,,4] | [B,,8B8B,8B,,,8] [E,4E4e4E,,4] |
[E,8e8E8E,,8] [B,4B4b4B,,4] | [B,8b8B8B,,8] [E4e'4e4E,4] | [E8e8e'8E,8] [=F4f4G,4b4d'4f'4B,4D4] | [=F6f6G,6b6d'6f'6B,6D6]e'd' c'bag |
=fedc ((3B2A2G2)((3F2E2D2) CB,A,G,^F, | G,8 [B,,,4G,4=F4D4B,,4] | [C,8G,8E8C8C,,8] [D4d4E,4^g4b4d'4G,4B,4] | [D6d6E,6^g6b6d'6G,6B,6]c'b agfe |
dcBA ((3^G2F2E2)((3D2C2B,2) A,G,F,E,^D, | E,8 [^G,,,4E,4D4B,4G,,4] | [A,,8E,8C8A,8A,,,8] [^A,4a4] | [^a6A,6]=ag =f^d=dc |
^A=AG=F ((3^D2=D2C2) ^A,4 | ^A,8 E4 | E8 [E4^A,4G4] | [^A,8E8G8] [E4G4^c4A,4] |
[^A,8^c8G8E8] [G4C4e4g4c4E4A,4] | [G8g8E8e8^A,8^c8C8] [F4f4] | [=F8f8] [^A,4E4e4^c4^f4C4F4] | [B,2F2D2f2d2B2][F2B,2D2] [F2B,2D2][F2B,2D2] [F2B,2D2][F2B,2D2] |
[D,2B,2F,2b2f2B2][B,2D,2F,2] [B,2D,2F,2][B,2D,2F,2] [B,2D,2F,2][B,2D,2F,2] | [^D,2A,2F2B2F,2][A,2F,2D,2] [E,2c2C2G,2][E,2G,2] [E,2G,2][E,2G,2] | [E,2B,2c2G2G,2][E,2G,2B,2] [F,2^d2D2A,2][A,2F,2] [A,2F,2][A,2F,2] | [F,2C2^d2A2A,2][C2A,2F,2] [G,2e2E2B,2][B,2G,2] [B,2G,2][B,2G,2] |
[B,2e2B2^G,2][D2B,2G,2] [^C2G,2=f2F2B,2][C2B,2G,2] [C2B,2G,2][C2B,2G,2] | [^C2^G2B2e2][C2=f2B2G2] [C2f2B2G2][C2f2B2G2] [C2f2B2G2][C2f2B2G2] | [^C2^G2B2=f2][C2f2B2G2] [C2f2B2G2][C2f2B2G2] [C2f2B2G2][C2f2B2G2] | [E2e'2f2f'2^a2^c'2A,2F,2C2][F,2e'2g2g'2a2c'2C2E2A,2] [F,2e'2g2g'2a2c'2C2E2A,2][F,2e'2g2g'2a2c'2C2E2A,2] [F,2e'2g2g'2a2c'2C2E2A,2][F,2e'2g2g'2a2c'2C2E2A,2] |
[F,2e'2g2g'2^a2^c'2C2E2A,2][E2e'2g2g'2a2c'2A,2F,2C2] [E2e'2g2g'2a2c'2A,2F,2C2][E2e'2g2g'2a2c'2A,2F,2C2] [E2e'2g2g'2a2c'2A,2F,2C2][E2e'2g2g'2a2c'2A,2F,2C2] | [B,,gg']DF,D [^C,ff']EF,E D,FF,F | B,,DF,D [^C,bB]EF,E D,FF,F | B,,DF,D [^C,f]EF,E D,FF,[FF] |
B,,DF,D [^C,B,B]EF,E D,FF,F | E,DB,D [=F,G]DB,D [^F,F]DB,D | [F,F]DB,D F,DB,D [F,^A]E^CE | [B,,B]D[F,g]D [^C,g']EF,E D,F[F,f']F |
B,,D[F,f']D [^C,c']EF,E D,F[F,b]F | B,,D[F,b]D [^C,g]EF,E D,F[F,f]F | B,,D[F,f]D [^C,c]EF,E D,F[F,B]F | [E,B]DB,D [=F,G]DB,D [^F,F]DB,D |
[F2F,2]D2 B,2G,2 [F,2F,,,2F,,2]^A,2 | [B,,,2B,,2B,2][^C,,2C,2] [D,,2D,2][E,,2E,2] [F,4F,,4] | [G,12B,12] | [C8G,8E,8E,,8] G4 |
[F,,8^C8E8^A,8F,8] F4 | [B,,,2B,,2B,2D2F2][^C,,2C,2] [D,,2D,2][E,,2E,2] [F,4F,,4] | [G,12B,12] | [C8G,8E,8E,,8] G4 |
[F,8^C8E8^A,8F,,8] F12 | G4 | [B,,8B,,,8B,8D8] F12 | G4 |
[E8F,,8^C8F,,,8] ^A4 | [B,,12B12B,12D12B,,,12] | [B,,12B12B,12D12B,,,12] | [B,,12B12B,12D12B,,,12] |
B12 | B12 | B8 [F2a2B2A2]B2 | [B2A2F2a2]B2 B2[B2g2] [F2B2f2A2]B2 |
[B2b2g2E2G2]B2 B2B2 [B2E2b2g2G2]B2 | [B2=F2D2^g2b2]B2 B2[B2a2] [B2F2b2g2D2]B2 | [E2C2c'2a2c2]E2 E2E2 [E2c'2c2]E2 | [E2C2A,2c'2c2]E2 E2[E2b2B2] [A,2E2A2a2C2]E2 |
[=F2d'2d2^G,2B,2]F2 F2F2 [F2G,2d2d'2B,2]F2 | [=F2^A,2^G,2d'2d2]F2 F2[F2c'2c2] [G,2F2d2d'2A,2]F2 | [^D2G,2^A,2d'2d2][G,2D2A,2] [G,2D2A,2][G,2D2A,2=d'2d2] [G,2^D2A,2d2d'2][G,2A,2D2] | [^A,2F,2^D2][F,2A,2] [F,2A,2][F,2A,2=D2] [F,2A,2^D2][F,2A,2] |
[F,0^D,0D0F0][D,2F,2] [D,2F,2][F,2D,2=D2=F2] [^D,2^F,2D2F2][D,2F,2] | [^D,2d2f2C,2][C,2D,2] [C,2D,2][C,2D,2=d2=f2] [C,2^D,2d2^f2][C,2D,2] | [C,2A,,2c2c'2][A,,2C,2] [A,,2C,2][A,,2C,2B2b2] [A,,2C,2c2c'2][A,,2C,2] | [C,2A,,2F,,2A,2][A,,2C,2F,,2] [A,,2C,2F,,2][A,,2C,2F,,2^G,2] [A,,2C,2F,,2A,2][A,,2C,2F,,2] |
[C,2F,,2^D,,2A,,2f'2f2][A,,2F,,2D,,2C,2] [A,,2F,,2D,,2C,2][A,,2F,,2D,,2C,2=f'2f2] [A,,2^F,,2D,,2C,2f'2f2][A,,2D,,2F,,2C,2] | [C,2F,,2D,,2A,,2A2c2F2][A,,2F,,2D,,2C,2] [A,,2F,,2D,,2C,2][A,,2F,,2D,,2C,2F2A2B2] [A,,2F,,2D,,2C,2F2A2c2][A,,2D,,2F,,2C,2] | [G,,,2G,,2=F2^G2B2][=G,,2G,,,2] [G,,,2G,,2][f2B2A2] [e4c4G4] | [^d2A2F2][G,,,2G,,2] [G,,2G,,,2][d2A2F2] [=d4B4=F4] |
[^c2G2E2][G,,,2G,,2] [G,,2G,,,2][=c2G2^D2] [B4G4=D4] | [^A2E2^C2][G,,2G,,,2] [=A2E2=C2][G,,2G,,,2] [^G2^D2C2][=G,,2G,,,2] | G2[G,,,2G,,2] F2[A,,,2A,,2] =F2[B,,,2B,,2] | E2[C,,2C,2] ^D2[^C,2C,,2] =D2[D,2D,,2] |
^C2[^D,2D,,2] =C2[E,2E,,2] B,2[=F,2F,,2] | ^A,2[F,,2F,2] =A,2[G,,2G,2] ^G,2[=G,,2G,2] | ^G,2=G,2 G8 | =F8 [F4B,4G4] |
[E4C4] [C4c4] [^C4A4] | [D4G4] [=F4A,4] [^A,4F4] | B,4 [G,4d4] [D4=F4] | ^C4 [=C4E4] [E^A,C]Gce |
[=fA,]cAF [AF,F]cfa [a^F,D]dad' | [aG,C]da^a [gD,A,]dga [g=F,A,]^cga | [gE,]cg^a [eG]ceg [gC,A,]egc' | [g=F,]egc' [aA,F,]fac' [aA,,]fac' |
a=fad' [aD,]fad' [^aA,,]fad' | [bA,,]=fbd' [b^G,,]ebd' [bG,,]ebd' | bebd' [bE,]ebd' [b^G,,]ebd' | [bA,,]ebc' [aE,]eac' [aA,,]eac' |
[aB,,]dab [^gE,]dgb [gB,,]dgb | [^gC,]cgb [gE,]aca [gC,]aca | [a^D,]Bab [aB,]Bab [aD,]Bab | [aE,]Bab [aC]Bab [aE,]Bab |
[aF,]Bab [a^D]Bab [aF,]Bab | [gG,]Bgb [gE]Bgb [bG,]ebe' | [c'A,]ec'e' [c'F]ec'e' [c'A,]ec'e' | [^c'^A,]gc'e' [c'G]gc'e' [c'A,]gc'e' |
[bB,]gbe' [g'E]f'e'^d' [e'G]d'e'f' | [g'B]f'e'^d' [g'e]f'e'd' [g'g]f'e'd' | g'f'e'^d' [ge']fed e4 | g'2f'2 [g2e'2][f2^d'2] [e2e'2]d2 |
[e4g'4] f'4 [g4e'4] | [f4^d'4] [e4e'4] d4 | [e8g'8] f'4 | g4 e'4 f4 |
G4 e4 F2E2 | [E8G8] F2E2 | E4 G4 [F2G2]E2 | [E2F2]E2 E2G2 F2[g2E2] |
f2[e2g'2] f'2[g2e'2] f2[e2G2] | F2[G,2E2] F,2E,2 [E,4E4B,4G4E,,4G,4] | [G4E,4E4G,4B,4E,,4]>F4 [E,,4E4G,4E,4] | [D,8A8A,8F8D,,8] [D4F4A4] |
[D4F4A4C4]>[B,4D4G4] [C4A4F4A,4] | [B,8B8G8G,8] [G,,4G4D4B4G,4B,4] | [B4G,4G4D4B,4G,,4]>A4 [G,,,4G4B,4G,,4] | [F,,8^c8C8^A8F,,,8] [F4A4c4] |
[F4^A4^c4E4]>[D4F4B4] [E4c4A4C4] | [D8d8B8B,8] d4 | [D4B,4G4] g4 [e4E4] | [d4A,4] c4 [^D4c4F4] |
D4 a4 [F4c4] | G4 B4 [G,4B4] | E,4 G4 [E4G4] | C4 E4 [E4C,4] |
C,,4 G,4 E,4 | [B,,,8^D,8B,8B,,8] [e'4e4] | [G,,4B,4E,4]>[^d'4d4] [e'4e4] | [A,,8E8F8C8] [^A,,4E4F4^C4] |
[F4B,,4^D4A,4]>G4 A2G2 | [F4C,4G,4] E4 [B,,2e'2e2]A,,2 | [G,,4B,4E,4]>[^d'4d4] [^G,,4e4e'4] | [A,,8E8F8C8] [^A,,4E4F4^C4] |
[F4B,,4A,4^D4B,,,4]>A4 G2F2 | [E,,8G,8E8E,8] [G,,4G,4G4G,,,4] | [G,,8G8G,8G,,,8] [C,4C4c4C,,4] | [C,8c8C8C,,8] [G,4G4g4G,,4] |
[G,8g8G8G,,8] [C4c'4c4C,4] | [C8c8c'8C,8] [G4D4d'4=f'4f4F4B,4] | [G6D6d'6=f'6f6F6B,6]e'd' c'bag | =fedc ((3B2A2G2)((3F2E2D2) CB,A,G,^F, |
G,8 [B,,,4G,4=F4D4B,,4] | [C,8G,8E8C8C,,8] [D4d4E,4^g4b4d'4G,4B,4] | [D6d6E,6^g6b6d'6G,6B,6]c'b agfe | dcBA ((3^G2F2E2)((3D2C2B,2) A,G,F,E,^D, |
E,8 [^G,,,4E,4D4B,4G,,4] | [A,,8E,8C8A,8A,,,8] [A,4c'4=F,,4C,4] | [c'6=F,,6A,6C,6]^a=a gfed | c^A=AG FEDC^A, [=A,4C,4=F,,4] |
[C,8=F,,8A,8] [F,,4A,4^D4C,4] | [C,8A,8^D8=F,,8] [A,4^F,,4D4A4C,4] | [C,8F,,8^D8A8A,8] [A,4A4d4D4C,4F,,4] | [C,8^d8A8D8A,8F,,8] [A,4F,,4c4a4d4C,4c'4] |
[c'8a8c8^d8A,8B,,8F,,8] [b4d4B4a4] | [^a8^d8A8] [F,,4=A,4b4a4B4B,,4d4] | [G,,2G,2B,,2e2b2B2][B,2G,2] [B,2G,2][B,2G,2] [B,2G,2][B,2G,2] | [^G,2B,2e'2e2][B,2G,2] [B,2G,2][B,2G,2] [B,2G,2][B,2G,2] |
[^G,2d2e2B2B,2D2][D2B,2G,2] [A,2=F2c2f2C2][A,2C2] [A,2C2][A,2C2] | [A,2=f2c2e2C2E2][A,2C2E2] [B,2^G2d2g2D2][D2B,2] [D2B,2][D2B,2] | [B,2E2d2^g2e2D2G2][E2D2B,2] [C2A2e2a2E2][E2C2] [E2C2][E2C2] | [E2a2e2C2][E2^C2] [E2e2^a2g2C2A2][E2C2] [E2C2][E2C2] |
[F2^c2e2a2][F2^a2e2c2] [F2a2e2c2][F2a2e2c2] [F2a2e2c2][F2a2e2c2] | [F2^c2e2^a2][F2a2e2c2] [F2a2e2c2][F2a2e2c2] [F2a2e2c2][F2a2e2c2] | [A2^D2b2B,2F2][B,2c''2A2c'2D2F2] [B,2c''2A2c'2D2F2][B,2c''2A2c'2D2F2] [B,2c''2A2c'2D2F2][B,2c''2A2c'2D2F2] | [B,2c''2A2c'2^D2F2][A2c''2B,2c'2F2D2] [A2c''2B,2c'2F2D2][A2c''2B,2c'2F2D2] [A2c''2B,2c'2F2D2][A2c''2B,2c'2F2D2] |
[E,c'c'']GB,G [F,bb']AB,A G,BB,B | E,GB,G [F,e'e]AB,A G,BB,B | E,GB,G [F,b]AB,A G,BB,[BB] | E,GB,G [F,Ee]AB,A G,BB,B |
A,GEG [^A,c]GEG [B,B]GEG | [B,B]GEG B,GEG [B,^d]AFA | [E,,e]G,[B,,c]G, [F,,c']A,B,,A, G,,B,[B,,b]B, | E,,G,[B,,b]G, [F,,f]A,B,,A, G,,B,[B,,e]B, |
E,,G,[B,,e]G, [F,,c]A,B,,A, G,,B,[B,,B]B, | E,,G,[B,,B]G, [F,,F]A,B,,A, G,,B,[B,,E]B, | [A,,E]G,E,G, [^A,,C]G,E,G, [B,,B,]G,E,G, | [B,2B,,2]G,2 E,2C,2 [B,,2B,,,2]^D,2 |
[E,,2E,2][F,,2F,2] [G,,2G,2][A,,2A,2] [B,4B,,4] | [C12E12] | [=F8C8A,8A,,8] c4 | [B,,8F8A8^D8B,8] B4 |
[E,,,2E,,2E2G2B2][F,,,2F,,2] [G,,,2G,,2][A,,,2A,,2] [B,,4B,,,4] | [C,12E,12] | [=F,8C,8A,,8A,,,8] C4 | [B,,8F,8A,8^D,8B,,,8] B,12 |
C4 | [B,,8E,,8E,8G,8] B,12 | C4 | [A,8B,,8F,8B,,,8] ^D4 |
[E,,8E8G,8] c4 | [F8B,8A8^D8B,,8] B12 | c4 | [E,8G8E8B,8] B12 |
c4 | [A8B,8F8B,,8] ^d4 | [E,8e8G8] c'4 | [a8B8^d8f8] b12 |
c8 | B4 [F4f4^d4A4] | [G4e4g4E4]>[F4^d4f4A4] [G4e4] | [A8D8a8f8F8] [d4f4a4] |
[d4f4a4c4]>[B4d4g4] [c4f4a4A4] | [B8b8g8G8] [g4b4] | [g4b4]>[f4a4] [e4b4g4] | [^d8a8c'8] [e'4e4] |
[E,4G,,4B,4]>[^d'4d4] [e'4e4] | [A,,8C8E8F8] [^A,,4F4E4^C4] | [F4B,,4^D4A,4]>G4 A2G2 | [F4C,4G,4] E4 [B,,2B,,,2e'2e2][A,,2A,,,2] |
[G,,4G,,,4]>[^d'4d4] [^G,,4e4e'4G,,,4] | [A,,8F8C8E8A,,,8] [^A,,4^C4F4E4A,,,4] | [F4B,,4A,4^D4B,,,4]>A4 G2F2 | [E,,4E4G,4] |

'''



# music_abc = header + music_abc

In [6]:
from music21 import *
abcScore = converter.parse(music_abc, format='abc')
mf = midi.translate.streamToMidiFile(abcScore)
mf.open('./midi_epoch20-03.mid', 'wb')
mf.write()
mf.close()